In [2]:
import pandas as pd
import numpy as np
from pygeotile.tile import Tile
from tqdm import tqdm_notebook

import plotly.express as px
import geojson
np.set_printoptions(suppress=True)
import plotly.graph_objects as go

from scipy.interpolate import griddata
import matplotlib.pyplot as plt
import pickle

%matplotlib inline


# Moscow coordinates
MAX_LAT = 55.950516
MIN_LAT = 55.540331
MIN_LON = 37.248575
MAX_LON = 37.932603


# Spb coordinates
# MAX_LAT = 60.068108
# MIN_LAT = 59.674527
# MAX_LON = 30.749038
# MIN_LON = 30.022010
token = 'pk.eyJ1IjoiZXZnZW5paWdhdnJpbGluIiwiYSI6ImNrMG50N3ptdjAzNW8zbm8wZzVmaXpzcWoifQ.LMSJohnSoBN-6YlAgKPO0w'
px.set_mapbox_access_token(token)

pd.options.display.float_format = '{:.2f}'.format

In [3]:
# Loading geocoded cian flat coordinates

with open('../combined_files/cian_coords.pickle.pkl', 'rb') as handle:
    dict_parsed_flats = pickle.load(handle)
    
d = pd.DataFrame.from_dict(dict_parsed_flats, orient='index')
d = d.reset_index()
d = d[d.columns[:2]]

d.columns = ['id', 'coord']
d['len'] = d['coord'].apply(lambda x: len(x))
d = d[d['len'] > 7].copy()
d['lat'] = d['coord'].apply(lambda x: x.split(' ')[1]).astype(float)
d['lon'] = d['coord'].apply(lambda x: x.split(' ')[0]).astype(float)
df_cian_id = d[['id', 'lat', 'lon']]

df_cian_id.head()

,id,lat,lon
0,212740701,55.77,37.73
1,207375181,55.75,37.72
2,215559357,55.74,37.71
3,215611880,55.75,37.72
4,201652911,55.75,37.71


In [4]:
# load cian flats,merge coordinates by id
df1 = pd.read_csv('../combined_files/cian_flats.csv', sep = ';')
df1 = df1.drop_duplicates(subset=['id'])
df1 = df1[df1.columns[1:]]

# it's a parsing bug - all flats came as an int property with missing comma
# TODO: fix bug
df1['square'] = df1['square'].apply(lambda x: str(x)[:2]).astype(int)
df1 = pd.merge(df1, df_cian_id, left_on = 'id',right_on = 'id')
df1['mean_price'] = df1['price'] / df1['square']
df1 = df1[(df1['mean_price'] > df1['mean_price'].quantile(0.003)) &
          (df1['mean_price'] < df1['mean_price'].quantile(0.997))]
df1 = df1.dropna()
df1 = df1[['id', 'lat','lon', 'price', 'square', 'mean_price']]
df1 = df1[df1['square'] > 30]

In [5]:
# load avito flats
df = pd.read_csv('../combined_files/avito_flats.csv', sep = ';')

df = df[df.columns[1:]]
df['mean_price'] = df['price'] / df['square']
df = df[(df['mean_price'] > df['mean_price'].quantile(0.003)) & (df['mean_price'] < df['mean_price'].quantile(0.997))]
df = df.dropna()
df = df[['id', 'lat','lon', 'price', 'square', 'mean_price']]
df['mean_price'].min(), df['mean_price'].max()

(19230.75, 920613.7424949966)

In [6]:
def clip_dataframe(df, min_lat=MIN_LAT, max_lat=MAX_LAT, min_lon=MIN_LON, max_lon=MAX_LON):
    """ Clip by bounding box """
    df_clip = df[(df.lat >= MIN_LAT) & (df.lat <= MAX_LAT) & (df.lon >= MIN_LON) & (df.lon <=MAX_LON)]
    return df_clip

In [7]:
df_msk = clip_dataframe(df)
df_msk.shape

(27665, 6)

In [8]:
def make_interpolation(df, bl_step = 0.001):
    xi = np.arange(MIN_LAT, MAX_LAT, bl_step)
    yi = np.arange(MIN_LON, MAX_LON, bl_step)
    xi,yi = np.meshgrid(xi,yi)
    
    x = df['lat']
    y = df['lon']
    z = df['mean_price']
    zi = griddata((x,y),z,(xi,yi),method='linear')
    
    lat = pd.DataFrame(xi.reshape(-1,1), columns = ['lat'])
    lon = pd.DataFrame(yi.reshape(-1,1), columns = ['lon'])
    price = pd.DataFrame(zi.reshape(-1,1), columns = ['mean_price'])
    df_interpolated = pd.concat([lat, lon, price],axis = 1, sort=False)
    df_interpolated = df_interpolated.dropna()
    return df_interpolated

In [9]:
def create_z_scale_tile(dataframe, lat_column = 'lat', lon_column ='lon', z_scale = 12):
    """ Creating geo quad index tile of z_scale """    
    tiles = []
    d_tile_center = {}
    # TODO: try to avoid this loop. Vectorize
    for ix, row in dataframe.iterrows():
        if np.isnan(row[lat_column]):
            tiles.append(Tile.for_latitude_longitude(0,0,0).quad_tree)
        else:
            tile = Tile.for_latitude_longitude(row[lat_column], row[lon_column], z_scale)
            tiles.append(tile.quad_tree) 
            bounds = tile.bounds
            d_tile_center[tile.quad_tree] = (np.mean([bounds[0].latitude, bounds[1].latitude]),
                                   np.mean([bounds[0].longitude,bounds[1].longitude]))
    dataframe[f'tile_z_{z_scale}'] = tiles
    return dataframe, d_tile_center
    

def create_tiles(dataframe, list_scales, lat_column = 'lat', lon_column ='lon'):
    """ Creating geo quad index tile of z_scale """
    d_tile_centers = {}
    for scale in tqdm_notebook(list_scales):
        dataframe, tile_centers = create_z_scale_tile(dataframe, lat_column, lon_column, scale)
        d_tile_centers.update(tile_centers)
        
    df_tile_centers = pd.DataFrame(d_tile_centers).T.reset_index()
    df_tile_centers.columns = ['tile_z_scale', 'c_lat', 'c_lon']    
        
    return dataframe, df_tile_centers

In [ ]:
df_interpolated = make_interpolation(df_msk)

df_interpolated, df_tile_centers = create_tiles(df_interpolated, list_scales=[15, 16, 17, 18],
                                  lat_column = 'lat', lon_column ='lon')

In [ ]:
def create_mean_tile_price(df, df_tile_centers, z_scale):
    """ Mean price in tile of z_scale """
    column = f'tile_z_{z_scale}'
    
    df_z = df[['lat','lon', 'mean_price', column]]
    
    df_z = pd.merge(df_z, df_tile_centers, left_on = column, right_on = 'tile_z_scale')
    df_z['mean_tile_price'] = df_z.groupby(column)['mean_price'].transform(np.mean)
    df_z['cnt_flats'] = df_z.groupby('tile_z_scale')['tile_z_scale'].transform('count')
    return df_z


def plot_scatter(df_tile):
    
    df_tile_u = df_tile[['c_lat','c_lon','mean_tile_price','cnt_flats']].drop_duplicates()
    bins = pd.qcut(df_tile_u['mean_tile_price'], q=50, precision=1)
    df_tile_u['bin'] = bins.apply(lambda x: np.mean([x.left, x.right])).astype(int)
    
    plt.figure(figsize=(25, 25))
    plt.scatter(df_tile_u['c_lat'], df_tile_u['c_lon'], 
                s = np.log(df_tile_u['cnt_flats'])*4,
                c = df_tile_u['bin'], 
                cmap = 'jet')
    plt.xlabel("lat")
    plt.ylabel("lon")
    plt.legend(loc='upper left')   
    
    
# def plot_map(df_tile, z_scale): 
    
#     column = f'tile_z_{z_scale}'
#     df_tile_u = df_tile[['c_lat','c_lon','mean_tile_price','cnt_flats', column]].drop_duplicates()
#     bins = pd.qcut(df_tile_u['mean_tile_price'], q=10, precision=1)
#     df_tile_u['bin'] = bins.apply(lambda x: np.mean([x.left, x.right])).astype(int)
#     df_tile_u['cnt_flats'] = np.log(df_tile_u['cnt_flats'])    
#     fig = px.scatter_mapbox(df_tile_u, lat="c_lat", lon="c_lon",  
#                             color="bin", size="cnt_flats",
#                             color_continuous_scale=px.colors.diverging.Tealrose, 
#                             zoom=10)
#     return fig


def plot_map(df_tile, z_scale): 
    
    column = f'tile_z_{z_scale}'
    df_tile_u = df_tile[['c_lat','c_lon','mean_tile_price','cnt_flats', column]].drop_duplicates()
    bins = pd.qcut(df_tile_u['mean_tile_price'], q=50, precision=1)
    df_tile_u['bin'] = bins.apply(lambda x: x.left).astype(int)    
    
    features=[]
    for ix, row in df_tile_u.iterrows():
        if row[column] != '':
            t = Tile.from_quad_tree(row[column])
            coords = [[(t.bounds[0][1], t.bounds[0][0]),
                       (t.bounds[0][1], t.bounds[1][0]),
                      (t.bounds[1][1], t.bounds[1][0]),  
                      (t.bounds[1][1], t.bounds[0][0]),
                     ]]

            features.append(
                    geojson.Feature(id=row[column], geometry=geojson.Polygon(coords),
                                    properties={'price': row["mean_tile_price"]}))
    
    fig = go.Figure(go.Choroplethmapbox(geojson=geojson.FeatureCollection(features),
                                        locations=df_tile_u[column],
                                        z=df_tile_u['bin'],
                                        colorscale=[[0, 'darkgreen'], 
                                                    [0.4, 'lime'], 
                                                    [0.5, 'yellow'],
                                                    [0.7, 'orange'],
                                                    [0.9, 'red'],
                                                    [1.0, 'darkred']],                                        
                                        marker_line_width=None,
                                        marker_opacity=0.2,
                                        ))
    fig.update_layout(mapbox_style="light", mapbox_accesstoken=token,
                      mapbox_zoom=7, mapbox_center = {"lat": MAX_LAT, "lon": MAX_LON})
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig


def visualize(df, z_scale = 20):
    df_z = create_mean_tile_price(df, df_tile_centers, z_scale)
    return plot_map(df_z, z_scale)

In [ ]:
visualize(df_interpolated, 17)